In [ ]:
################################################################################################
# 입력 폴더에 말뭉치들을 regular express 처리하는 예제
################################################################################################

import os

# 입력 폴더 말뭉치 파일명들을 리스트로 얻어옴
in_folder_path = '../../../korpora/moco/txt'
file_list = os.listdir(in_folder_path)
print(file_list)

# 출력 폴더 지정. 없으면 폴더 생성
out_folder_path = '../../../korpora/moco/re-txt'
os.makedirs(out_folder_path, exist_ok=True)

In [ ]:
#############################################################################
# regular express를 이용한 말뭉치 정재하기
#
#############################################################################
# 정규화할 인자들
alphabetP=True            # False=알바벳 제거
numberP=True             # False=숫자제거
punctuationP=False       # False=특수문자 (, . ? !) 제거
symbolP=False            # False=괄호((), [], {}, ], ')등 제거

# 4번이상 반복되는 단어 중복 제거 (remove_repeat_wordP=2 : 2번만 호출)
# => 예: 안녕안녕안녕안녕 => 안녕안녕, 안녕 안녕 안녕 안녕 => 그대로(*띄어쓰기한 단어는 중복제거 안됨)
remove_repeat_wordP=2     

# 4번이상 반복되는 문자 중복 제거 (nremove_repeat_charP=2 : 2번만 호출)
# => 예: aaaaa => aa, aaa aaa aaa aaa => 그대로(*띄어쓰기한 문자는 중복제거 안됨)
remove_repeat_charP=2

# 정규화할 입력 text가 있는 폴더
#in_txt_path = '../../../korpora/kowiki/kowiki-202206-nlp-corpus.txt'
#in_txt_path = '../../../korpora/moco/test-60.txt'

# 출력 text
#out_txt_path = '../../../korpora/kowiki/re-kowiki-202206-nlp-corpus.txt'

# 삭제할 최대 길이 문장
remove_min_len = 30 # 20이면 20보다 작은문장은 제거함

remove_max_len = 600 # 1000자 보다 큰 문장은 제거함

print_count = 100000 # 10000번째마다 문장 출력해봄

In [ ]:
# reqular 함수들 정의 
import re

doublespace_pattern = re.compile('\s+')
repeatchars_pattern = re.compile('(\w)\\1{3,}')
number_pattern = re.compile('[0-9]')
punctuation_pattern = re.compile('[,\.\?\!]')
symbol_pattern = re.compile('[()\[\]\{\}`]')
hangle_pattern = re.compile('[ㄱ-ㅎㅏ-ㅣ가-힣]')
alphabet_pattern = re.compile('[a-zA-Z]')

hangle_filter = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣]')
hangle_number_filter = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9]')
text_filter = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9,\.\?\!\"\'-()\[\]\{\}]')

##========================================================================
# 4번이상 반복되는 단어 중복 제거 (num_repeat=2 : 2번만 호출)
# => 예: 안녕안녕안녕안녕 => 안녕안녕, 안녕 안녕 안녕 안녕 => 그대로(*띄어쓰기한 단어는 중복제거 안됨)
repeatchars_patterns = [
    re.compile('(\w\w\w\w)\\1{3,}'),
    re.compile('(\w\w\w)\\1{3,}'),
    re.compile('(\w\w)\\1{3,}'),
    re.compile('(\w)\\1{3,}')
]

def normalizetoken(sentence, num_repeat=2):
    tokens = sentence.split()
    return ' '.join(_normalize_korean_token(token, num_repeat) for token in tokens)

def _normalize_korean_token(token, num_repeat=2):
    #print(token)
    #token = _normalize_emoji(token)
    token = _remove_repeat(token, num_repeat)
    return token

def _remove_repeat(token, num_repeat=2):
    if num_repeat > 0:
        for pattern in repeatchars_patterns:
            #print(pattern)
            token = pattern.sub('\\1' * num_repeat, token)
    return token
##========================================================================


def normalize(sentence, 
              alphabet=False, 
              number=False,
              punctuation=False, 
              symbol=False, 
              remove_repeat_word=0, 
              remove_repeat_char=0):

    sentence = text_filter.sub(' ', sentence)
    
    if not alphabet:
        sentence = alphabet_pattern.sub(' ', sentence)
    if not number:
        sentence = number_pattern.sub(' ', sentence)
    if not punctuation:
        sentence = punctuation_pattern.sub(' ', sentence)
    if not symbol:
        sentence = symbol_pattern.sub(' ', sentence)
    if remove_repeat_char > 0:
        sentence = repeatchars_pattern.sub('\\1' * remove_repeat_char, sentence)
        
    if remove_repeat_word > 0:
        sentence = normalizetoken(sentence, num_repeat=remove_repeat_word)
        
    return doublespace_pattern.sub(' ', sentence).strip()

In [ ]:
## yield 를 이용하여 generator 함수 정의    
def get_generator_corpus(data, max_len: int=100000):
    
    #dataset = data
    dataset = list(set(data)) # ****중복 문장 제거(*순서 유지 안함)
    
    for start_idx in range(0, len(dataset), max_len):
        samples = dataset[start_idx : start_idx + max_len]
        yield samples

In [ ]:
from tqdm.notebook import tqdm

for filename in file_list:
    in_txt_path = in_folder_path+'/'+filename  # 입력 말뭉치 파일 풀경로 
    out_txt_path = out_folder_path+'/'+filename# 출력 파일 풀경로 
    
    # 1.분석할 text 읽어오기(한줄씩 읽어오기)
    with open(in_txt_path, 'r', encoding='utf8') as f:
        #lines = f.read()
        lines = f.readlines()
        #print(lines)

    training_corpus = get_generator_corpus(lines)

    result=[]
    count = 0
    for lines in training_corpus:
        for line in tqdm(lines):

            #print(line)
            # false=제거함.
            clearline = normalize(line, 
                                  alphabet=alphabetP, number=numberP, punctuation=punctuationP, symbol=symbolP, 
                                  remove_repeat_word=remove_repeat_wordP, remove_repeat_char=remove_repeat_charP)

            clearline1=clearline.strip()
            count += 1
            if count % print_count == 0:  # print_count 째마다 출력해봄
                print(f'[{count}]{clearline1}')

            #if clearline1 not in result: # ****순서유지 중복 제거(*순서유지하는 경우 시간 오래걸림)

            # remove_min_len 보가 크고, remove_max_len 보다 작은 경우에만 저장
            clearline1_len = len(clearline1)
            if clearline1_len > remove_min_len and clearline1_len < remove_max_len: # 20자 이상인 경우에만 출력
                #print(clearline1)
                result.append(clearline1)

    # 중복 문장 제거(*순서 유지 안함)
    #result = list(set(clearlinelist))

    #print('\r\n===[out]====')
    print(f'*len:{len(result)}')
    #count = 0
    #for line1 in result:
    #    print(line1)
    #    count += 1
    #    if count > 10:
    #        break
            
   # 리스트를 파일로 저장
    with open(out_txt_path, 'w', encoding='utf8') as f:
        f.write('\n'.join(result)) 

In [2]:
# OUT 파일들을 병합하여 하나의 파일로 만듬.
import os
#out_folder_path = '../../../korpora/moco/re-txt'

# 입력 폴더 말뭉치 파일명들을 리스트로 얻어옴
file_list = os.listdir(out_folder_path)
print(file_list)

out_file = '../../../korpora/moco/re-txt/0-moco-corpus2-t.txt'

with open(out_file, 'w', encoding='utf8') as outfile:
    for filename in file_list:
        in_txt_path = out_folder_path+'/'+filename  # 출력 파일 풀경로 
        with open(in_txt_path, 'r', encoding='utf8') as file:
            for line in file:
                outfile.write(line)

['moco (1).txt', 'moco (10).txt', 'moco (100).txt', 'moco (101).txt', 'moco (102).txt', 'moco (103).txt', 'moco (104).txt', 'moco (105).txt', 'moco (106).txt', 'moco (107).txt', 'moco (108).txt', 'moco (109).txt', 'moco (11).txt', 'moco (110).txt', 'moco (111).txt', 'moco (112).txt', 'moco (113).txt', 'moco (114).txt', 'moco (115).txt', 'moco (116).txt', 'moco (117).txt', 'moco (118).txt', 'moco (119).txt', 'moco (12).txt', 'moco (120).txt', 'moco (121).txt', 'moco (122).txt', 'moco (123).txt', 'moco (124).txt', 'moco (125).txt', 'moco (126).txt', 'moco (127).txt', 'moco (128).txt', 'moco (129).txt', 'moco (13).txt', 'moco (130).txt', 'moco (131).txt', 'moco (132).txt', 'moco (133).txt', 'moco (134).txt', 'moco (135).txt', 'moco (136).txt', 'moco (137).txt', 'moco (138).txt', 'moco (139).txt', 'moco (14).txt', 'moco (140).txt', 'moco (141).txt', 'moco (142).txt', 'moco (143).txt', 'moco (144).txt', 'moco (145).txt', 'moco (146).txt', 'moco (147).txt', 'moco (148).txt', 'moco (149).txt'